In [ ]:
import numpy as np
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

# Load the YOLOv8 model
model = YOLO("runs/barrel/yolov8n_custom/weights/best.pt")

In [ ]:
# Define the path or URL to the image
image_path = "valid/images/PXL_20240705_130706817_jpg.rf.7fb0e54df807bb41d227ea5df12837c0.jpg" # should be test image, not valid image (used for testing as there is no test image)

# Run inference on the image
results = model.predict(image_path, save=True, imgsz=640, conf=0.85)

# Get the first result (assuming only one image was passed)
result = results[0]

# Visualize the detections on the image
result_image = result.plot()

# Convert the color space from BGR to RGB
rgb_image = result_image[:, :, ::-1]

# Display the image with detections
plt.imshow(rgb_image)
plt.axis('off')  # Hide axis
plt.show()

In [ ]:
# Extract keypoints for the barrel and the tank
# Assuming result.keypoints contains the keypoints in the order specified in your dataset
barrel_keypoints = result.keypoints[:3]  # muzzle, top, bottom
tank_keypoints = result.keypoints[3:]    # top-right, bottom-right, bottom-left, top-left, rear

# Define the keypoints for barrel and tank
barrel_points = {
    "muzzle": barrel_keypoints[0],
    "intersection": barrel_keypoints[1]
}

tank_points = {
    "top-right": tank_keypoints[0],
    "rear": tank_keypoints[4]
}

# Function to calculate the angle between two vectors
def calculate_angle(pt1, pt2, pt3, pt4):
    # Vector from pt1 to pt2 (barrel)
    vector1 = np.array(pt2) - np.array(pt1)
    # Vector from pt3 to pt4 (tank)
    vector2 = np.array(pt4) - np.array(pt3)

    # Calculate the angle between two vectors
    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector1, unit_vector2)
    angle = np.arccos(dot_product)

    # Convert from radians to degrees
    angle = np.degrees(angle)
    return angle

# Calculate the rotation angle between the barrel and the tank
angle = calculate_angle(barrel_points["muzzle"], barrel_points["intersection"], tank_points["top-right"], tank_points["rear"])

print(f"Rotation Angle between barrel and tank: {angle:.2f} degrees")

# Visualize the detections on the image
result_image = result.plot()

# Convert the color space from BGR to RGB
rgb_image = result_image[:, :, ::-1]

# Display the image with detections
Image.fromarray(rgb_image).show()